In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Fuente

In [0]:
institucion_educativa_df = spark.read.format('csv') \
                                     .option('header', 'true') \
                                     .option('inferSchema', 'true') \
                                     .load('/FileStore/tables/2023_04_11T04_36_02_388Zinstituciones_educativas.csv')

In [0]:
universidades_homolac_df = spark.read.format('csv') \
                                     .option('header', 'true') \
                                     .option('inferSchema', 'true') \
                                     .load('/FileStore/tables/universidades_krowdy.csv')

#Transformacion

In [0]:
universidades_homolac_df = universidades_homolac_df.withColumn("siglas_min", trim(lower(col('Siglas ')))) \
                                                   .withColumn("nombre_min", trim(lower(col('Nombre ')))) \
                                                   .withColumn("id_inei", trim(col('código INEI'))).drop('código INEI')

In [0]:
df_homologado = institucion_educativa_df.withColumn("value_homologada", trim(lower(regexp_replace(col('value'), "\s+", " "))))

In [0]:
df_array = df_homologado.select("value_homologada", array("cod_inei").alias("cod_inei"))

import json
lista_dict = df_array.collect()

# Convertir cada elemento de la lista en un diccionario de Python
diccionario = {}
for elem in lista_dict:
    cadena_json = json.dumps(elem.asDict())
    dict_elem = json.loads(cadena_json)
    diccionario[dict_elem["value_homologada"]] = dict_elem["cod_inei"]

In [0]:
def homologate_name(value_homologada):
    if value_homologada in diccionario:
        return diccionario[value_homologada][0]  # Reemplazar con el primer sinónimo/abreviatura encontrado
    else:
        return value_homologada

# Aplicar la homologación
homologate_udf = udf(homologate_name, StringType())
df_homologado = df_homologado.withColumn("synonym_name", homologate_udf(col("value_homologada"))) \
                             .drop('cod_inei')

In [0]:
data_homologada = df_homologado.join(universidades_homolac_df, universidades_homolac_df.id_inei == df_homologado.synonym_name,'left') \
                               .select('candidateId','value',col('Nombre ').alias('universidad_homologada')) \
                               .filter(col('Nombre ').isNotNull())

In [0]:
data_homologada.display()

candidateId,value,universidad_homologada
630633fa1e599d0009d6dee0,Ucv,Universidad César Vallejo S.A.C.
63063bda29da5b00089fb224,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.
630665b8594e18004af047fc,"I. E 00 María Auxiliadora, Cercado De Lima",Universidad María Auxiliadora S.A.C.
630665b8594e18004af04816,Universidad Privada Del Norte - Upn,Universidad Privada del Norte S.A.C.
630665b8594e18004af04825,Universidad Nacional Agraria La Molina,Universidad Nacional Agraria La Molina
630665b8594e18004af04835,Universidad Privada San Juan Bautista,Universidad Privada San Juan Bautista S.A.C.
630665b8594e18004af04842,Universidad Peruana De Ciencias Aplicadas (upc),Universidad Peruana de Ciencias Aplicadas S.A.C.
630665b8594e18004af0485d,Universidad De Ciencias Aplicadas,Universidad Peruana de Ciencias Aplicadas S.A.C.
630665b8594e18004af04864,Universidad Peruana De Ciencias Aplicadas - Upc,Universidad Peruana de Ciencias Aplicadas S.A.C.
630665b8594e18004af0489e,Universidad Cesar Vallejo,Universidad César Vallejo S.A.C.


#Transormacion Diccionario

In [0]:
unpivot = "stack(1, 'value',value) as (ex_colum, value)"

unpivot_data = data_homologada.select(col("universidad_homologada"), expr(unpivot)).distinct() \
                              .drop('ex_colum')


df_array_data = unpivot_data.groupBy("universidad_homologada").agg(collect_list("value").alias("value"))

# Crear la lista de diccionarios
lista_sinonimos = []
for row in df_array_data.collect():
    universidad = row.universidad_homologada
    sinonimos = row.value
    sinonimos_dict = {"nombre_universidad": universidad, "sinonimos": sinonimos}
    lista_sinonimos.append(sinonimos_dict)

# Imprimir la lista
print(lista_sinonimos)

[{'nombre_universidad': 'Universidad Nacional Agraria La Molina', 'sinonimos': ['Universidad Nacional Agraria La Molina', 'Universidad Nacional Agraria La Molina - La Molina', 'Unalm']}, {'nombre_universidad': 'Universidad de Ingeniería y Tecnología', 'sinonimos': ['Utec']}, {'nombre_universidad': 'Universidad Nacional de Jaén', 'sinonimos': ['Universidad Nacional De Jaén']}, {'nombre_universidad': 'Universidad Nacional del Altiplano', 'sinonimos': ['Universidad Nacional Del Altiplano']}, {'nombre_universidad': 'Universidad ESAN', 'sinonimos': ['Universidad Esan']}, {'nombre_universidad': 'Universidad San Ignacio de Loyola S.R.L.', 'sinonimos': ['Universidad San Ignacio De Loyola', 'Instituto San Ignacio De Loyola Isil', 'San Ignacio De Loyola - Isil', 'Universidad San Ignacio De Loyola(cpel)', 'San Ignacio De Loyola', 'Isil - Instituto San Ignacio De Loyola', 'Instituto San Ignacio De Loyola (isil)', 'Usil - San Ignacio De Loyola', 'Instituto San Ignacio De Loyola', 'Instituto Superior San Ignacio De Loyola (isil)', 'Univesidad San Ignacio De Loyola', 'San Ignacio De Loyola', 'Universidad San Ignacio De Loyola - Usil', 'Isil ( Instituto San Ignacio De Loyola)', 'Usil', 'Instituto San Ignacio De Loyola - Isil']}, {'nombre_universidad': 'Universidad Nacional de San Martín', 'sinonimos': ['Universidad Nacional De San Martín', 'Universidad Nacional De San Martin']}, {'nombre_universidad': 'Universidad Nacional del Callao', 'sinonimos': ['Universidad Nacional Del Callao', 'Universidad Nacional Del Callao Unac', 'Universidad Nacional Del Callao - Unac', 'Politecnico Nacional Del Callao', 'Universida Nacional Del Callao', 'Unac', 'Universidad Nacional Del Callao']}, {'nombre_universidad': 'Universidad Nacional Amazónica de Madre de Dios', 'sinonimos': ['Universidad Nacional Amazónica De Madre De Dios']}, {'nombre_universidad': 'Universidad Nacional de Ucayali', 'sinonimos': ['Universidad Nacional De Ucayali']}, {'nombre_universidad': 'Universidad Católica de Trujillo Benedicto XVI', 'sinonimos': ['Universidad Católica De Trujillo Benedicto Xvi']}, {'nombre_universidad': 'Universidad Tecnológica de los Andes', 'sinonimos': ['Universidad Tecnológica De Los Andes', 'Universidad Tecnologica De Los Andes']}, {'nombre_universidad': 'Universidad de Huánuco', 'sinonimos': ['Universidad De Huánuco', 'San Juan Bosco De Huanuco', 'Universidad De Huánuco - Udh', 'Juan Bosco De Huánuco', 'Universidad De Huanuco', 'Juan Bosco De Huanuco']}, {'nombre_universidad': 'Universidad Nacional de Piura', 'sinonimos': ['Universidad Nacional De Piura', 'Universidad Nacional De Piura - Unp', 'Escuela Técnica Superior De La Universidad Nacional De Piura', 'Escuela Tecnica Superior De Universidad Nacional De Piura']}, {'nombre_universidad': 'Universidad Nacional Agraria de la Selva', 'sinonimos': ['Universidad Nacional Agraria De La Selva - Unas', 'Universidad Nacional Agraria De La Selva']}, {'nombre_universidad': 'Universidad Nacional de San Agustín de Arequipa', 'sinonimos': ['Universidad Nacional De San Agustín De Arequipa', 'Universidad Nacional De San Agustin De Arequipa', 'Universidad San Agustín De Arequipa']}, {'nombre_universidad': 'Universidad Nacional de Frontera', 'sinonimos': ['Universidad Nacional De Fronteras', 'Universidad Nacional De Frontera']}, {'nombre_universidad': 'Universidad Señor de Sipán S.A.C.', 'sinonimos': ['Universidad Señor De Sipán - Uss', 'Universidad Señor De Sipan', 'Universidad Privada Señor De Sipan', 'Universidad Señor De Sipán']}, {'nombre_universidad': 'Universidad Privada de Tacna', 'sinonimos': ['Upt', 'Universidad Privada De Tacna - Upt', 'Universidad Privada De Tacna']}, {'nombre_universidad': 'Universidad Marcelino Champagnat', 'sinonimos': ['Universidad Marcelino Champagnat']}, {'nombre_universidad': 'Universidad Católica San Pablo', 'sinonimos': ['Universidad Católica San Pablo', 'Universidad Catolica San Pablo']}, {'nombre_universidad': 'Universidad Peruana Cayetano Heredia', 'sinonimos': ['José Cayetano Heredia